## Etapa 0 - Preparação
    - Bloco reservado para instalação do pyspark no ambiente.
    - Se já tiver instalado, pode avançar esta parte!

In [ ]:
pip install pyspark

## Etapa 1 - Iniciar sessão do Spark

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName(" Execução do trabalho inyterativo")\
  .getOrCreate()

spark.version

## Etapa 2 - Carregar os arquivos

In [ ]:
df_titles = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Spark/title_basics.tsv', header=True,
inferSchema=True, sep='\t')

df_ratings = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Spark/title_ratings.tsv', header=True,
inferSchema=True, sep='\t')

In [ ]:
df_titles.printSchema()
df_titles.show()

In [ ]:
df_ratings.printSchema()
df_ratings.show()

## Etapa 3 - Levantamento dos dados

In [ ]:
# Questão 1
# Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?
df_titles.filter(df_titles.titleType.isin(['movie', 'tvMovie']) & (df_titles.startYear == 2015)).count()

In [ ]:
# Questão 2
# Qual o gênero de títulos mais frequente?

from pyspark.sql.functions import desc
titulo_mais_frequente = df_titles.groupBy("genres").count().orderBy(desc("count")).first()

In [ ]:
# Relacionamento das tabelas
df_joined = df_titles.join(df_ratings, on='tconst', how='inner')

In [ ]:
# Questão 3
# Qual o gênero com a melhor nota média de títulos?

# Registre o DataFrame df_joined como uma tabela temporária
df_joined.createOrReplaceTempView("joined")

# Execute a consulta SQL para calcular a média das classificações por gênero
best_genre = spark.sql("""
    SELECT genres, AVG(averageRating) as avg_rating
    FROM joined
    GROUP BY genres
    ORDER BY avg_rating DESC
    LIMIT 1
""")

# Exiba o gênero com a melhor nota média
best_genre.show()

In [ ]:
# Questão 4
# Qual o vídeo game do gênero aventura mais bem avaliado em 2020?

spark.sql("""
    SELECT primaryTitle, genres, averageRating
    FROM joined
    WHERE titleType = "videoGame"
    AND startYear = "2020"
    AND genres LIKE "%Adventure%"
    ORDER BY averageRating DESC
    LIMIT 1
""").show()

In [ ]:
# Questão 5
# Quantos títulos de filmes diferentes existem?

df_titles.select('primaryTitle').distinct().count()

In [ ]:
# Questão 6
# Qual a duração média dos filmes com conteúdo adulto?

df_titles.filter(df_titles.isAdult == 1).describe().show()

In [ ]:
# Questão 7
# Quantos filmes têm o título atual (“primary”) diferente do título original?

df_titles.filter(df_titles.primaryTitle != df_titles.originalTitle).count()

In [ ]:
# Questão 8
# Qual o filme que tem o nome mais longo?

from pyspark.sql.functions import length

df_titles.withColumn('titleLength', length(col('primaryTitle'))) \
                           .orderBy(length(col('primaryTitle')), ascending=False) \
                           .limit(1).show()

In [ ]:
# Questão 9
# Qual filme tem a maior quantidade de votos?

df_ratings.orderBy('numVotes', ascending=False).limit(1).show()

In [ ]:
# Questão 10
# Qual é a menor nota média de um filme?

df_ratings.orderBy('averageRating', ascending=True).limit(1)